In [70]:
import pandas as pd
import glob 
import os 
import numpy as np
import re
import bs4 as bs
import urllib.request
import pandas as pd
pd.set_option('max_colwidth', 8000)
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import svm


In [78]:
def aggr_files():
    df = pd.DataFrame()
    
    path = r'raw_data' # use your path

    all_files = glob.glob(os.path.join(path, "*.xls"))
    all_files1 = [f.replace(r".xls",".txt") for f in all_files]  ## replacing the file extension with .txt
    df_from_each_file = (pd.read_csv(f, delimiter = "\t") for f in all_files) ## reading the .txt files using csv reader
    df   = pd.concat(df_from_each_file, ignore_index=True)  ## concatenating all the individual files
#     for x in range(2,13):
#         temp_df = pd.read_excel('raw_data/nieuwe_leden_20210723 ('+str(x)+').xls')
#         df = pd.concat([df, temp_df])
    return df

def drop_private_cols(df):
    df = df.drop(['Heb je je zwemdiploma A?','Anders, namelijk.1','Wat is het fanatiekst dat je hebt gesport?',
                 'Indien je niet in Amsterdam woont, ben je op zoek naar een kamer in Amsterdam?',
                 'Op Skøll eten veel mensen vegetarisch, pas je voorkeur hieronder aan als je toch wel vlees wilt eten',
                 'Namelijk', 'Wanneer verwacht je af te studeren?','Instituut',
                 'Wanneer ben je begonnen met je studie?','Heb je medische bijzonderheden waar wij rekening mee moeten houden?',
                 'Heb je allergieën waar wij rekening mee moeten houden?','Waar ken je Skøll van?',
                  'Heb je je naast sport op een ander gebied buitengewoon ingezet?',
                  'Zo ja, bij welke vereniging?','Hoeveel jaar heb je je langst beoefende sport beoefend?'
                 ], axis = 1)
    return df

def transform_df(df):
    df['Geslacht'].replace('Vrouw',0,inplace=True)
    df['Geslacht'].replace('Man',1,inplace=True)
    
    df['Heb je eerder geroeid?'].replace('Nee',0,inplace=True)
    df['Heb je eerder geroeid?'].replace('Ja',1,inplace=True)
    
    df['Gewicht (in kg)'] = df['Gewicht (in kg)'].str.extract('(\d+)')
    df['Lengte (in cm)'] = df['Lengte (in cm)'].str.extract('(\d+)')
    
    df['Ik wil naar Skøll voor'].fillna('no', inplace=True)
    df['skollvoorroeien'] = np.where(df['Ik wil naar Skøll voor'].str.contains('top_rowing'), 1, 0)
    df.drop(['Ik wil naar Skøll voor'], axis = 1, inplace = True)
    
    df['Zo ja, hoe lang?'].fillna('no', inplace=True)
    df['1+ jaar al geroeid'] = np.where(df['Zo ja, hoe lang?'].str.contains('jaar'), 1, 0)
    df.drop(['Zo ja, hoe lang?'], axis = 1, inplace = True)
    
    df['Hoe vaak kan roeien'] = df['Wanneer kun je roeien?'].str.count('ja').fillna(0)
    df.drop(['Wanneer kun je roeien?'], axis = 1, inplace = True)
    
    df['roeimotivatie'] = np.where(df['Waarom zou jij goed bij Skøll passen?'].str.contains('roei|sport|hard|wedstrijd'), 1, 0)
    df.drop(['Waarom zou jij goed bij Skøll passen?'], axis = 1, inplace = True)

    df['Welke sporten heb je beoefend?'] = df['Welke sporten heb je beoefend?'].str.split(',').str.len()
    df['Anders, namelijk'] = df['Anders, namelijk'].fillna('nothing')
    df['Anders, namelijk'] = df['Anders, namelijk'].str.split('(,|and)').str.len()
    df['hoeveel sporten vroeger beoefend'] = pd.to_numeric(df['Welke sporten heb je beoefend?']) + pd.to_numeric(df['Anders, namelijk']) - 1
    df.drop(['Anders, namelijk', 'Welke sporten heb je beoefend?'], axis = 1, inplace = True)
    
    mapdict1 = {'2 à 3 keer per week':2.5
               ,'Minder dan 2 keer per week':1
               ,'3 à 4 keer per week':3.5
               ,'Meer dan 4 keer per week':5
               ,'? keer per week': np.nan }
    df['# van plan te gaan roeien'] = df['Hoe vaak ben je van plan te gaan roeien per week?'].map(mapdict1)
    mapdict2 = {'2 à 3 keer per week':2.5,
        '3 à 4 keer per week':3.5,
        '4 à 5 keer per week':4.5,
        '? keer per week':0,
        'Meer dan 5 keer per week':6,
        'Minder dan 2 keer per week':1}
    df['# al geroeid'] = df['Zo ja, hoe vaak per week?'].map(mapdict2)
    df.drop(['Hoe vaak ben je van plan te gaan roeien per week?', 'Zo ja, hoe vaak per week?'], axis = 1, inplace = True)

    df['Wat is je beste sportprestatie?'] = df['Wat is je beste sportprestatie?'].fillna('nothing')
    df['sportroeiprestatie'] = np.where(df['Wat is je beste sportprestatie?'].str.contains('roei'), 1, 0)
    df.drop(['Wat is je beste sportprestatie?'], axis = 1, inplace = True)

    return df

In [79]:
df = pd.read_csv('raw_data/training_data.csv')
df = drop_private_cols(df)
df = transform_df(df)

In [80]:

# with open("raw_data/LedenMetBootjes.html", encoding='utf-8') as fp:
#     soup = bs.BeautifulSoup(fp, 'html.parser')

# table = soup.find(id='toolkit-table', attrs={'class':'wp-list-table widefat fixed posts'})

# table_rows = table.find_all('tr')

# l = []
# for tr in table_rows:
#     td = tr.find_all('td')
#     row = [tr.text for tr in td]
#     l.append(row)
# bootjes = pd.DataFrame(l, columns=["Name", "SaxoId", 'WpId', 'Boten'])
# bootjes.dropna(axis=0, inplace = True)
# # bootjes['Boten'] = bootjes['Boten'].str.split(',')
# # bootjes = bootjes[bootjes['Boten'].str.len() != 1]
# bootjes.shape
# bootjes['Name'] = bootjes['Name'].apply(hash)
# bootjes.to_csv('raw_data/people_with_boats.csv', index=False)
# bootjes.shape
bootjes = pd.read_csv('raw_data/people_with_boats.csv')
bootjes['Boten'] = bootjes['Boten'].str.split(',')
bootjes = bootjes[bootjes['Boten'].str.len() != 1]
bootjes.dropna(axis=0, inplace = True)
bootjes.columns


Index(['Name', 'SaxoId', 'WpId', 'Boten'], dtype='object')

In [81]:
# bootjes['TopC4'] = np.where(bootjes['Boten'].astype(str).str.contains('TopC'), 1, 0)
# bootjes['Talenten'] = np.where(bootjes['Boten'].astype(str).str.contains('Talenten '), 1, 0)
bootjes['Club'] = np.where(bootjes['Boten'].astype(str).str.contains('Club'), 1, 0)
bootjes['EerstejaarsWed'] = np.where(bootjes['Boten'].astype(str).str.contains('Eerstejaars'), 1, 0)
bootjes['MiddengroepWed'] = np.where(bootjes['Boten'].astype(str).str.contains('Middengroep'), 1, 0)
bootjes['Ouderejaars'] = np.where(bootjes['Boten'].astype(str).str.contains('Ouderejaars'), 1, 0)
bootjes['OudeVier'] = np.where(bootjes['Boten'].astype(str).str.contains('Oude Vier'), 1, 0)

bootjes['goldLabel'] = np.where(bootjes.iloc[:, 4:].any(axis='columns'), 1, 0)
# indb = bootjes[bootjes.iloc[:, 4:].any(axis='columns')].index.tolist() # select only the rowers who have done at least one of the above
label = bootjes.loc[:, ['Name', 'goldLabel']]
print(label.shape)
label.head()

(560, 2)


,Name,goldLabel
0,-5456831734576246383,0
1,-7503962078928739405,0
3,4273243590570684613,1
4,3131485771293917772,1
6,-2991480449040189023,0


In [82]:
enddf = pd.merge(label, df, how='inner',left_on='Name', right_on='Naam')
enddf.drop(['Naam', 'Name'], axis = 1, inplace = True)
enddf['Gewicht (in kg)'] = enddf['Gewicht (in kg)'].astype(int)
enddf['Lengte (in cm)'] = enddf['Lengte (in cm)'].astype(int)
enddf.fillna(0, inplace = True)
print(enddf.columns)

scale_data = False
if scale_data:
    allcols = enddf.columns
    x = enddf.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    enddf = pd.DataFrame(x_scaled)
    enddf.columns = allcols
    
else:
    enddf.drop(['Lengte (in cm)','Gewicht (in kg)'],axis = 1,inplace = True) # gewicht and lengte appears to work negatively on the prediction score

dffemale = enddf[enddf['Geslacht'] == 0].drop(['Geslacht'], axis = 1)
dfmale = enddf[enddf['Geslacht'] == 1].drop(['Geslacht'], axis = 1)

Index(['goldLabel', 'Geslacht', 'Heb je eerder geroeid?', 'Gewicht (in kg)',
       'Lengte (in cm)', 'skollvoorroeien', '1+ jaar al geroeid',
       'Hoe vaak kan roeien', 'roeimotivatie',
       'hoeveel sporten vroeger beoefend', '# van plan te gaan roeien',
       '# al geroeid', 'sportroeiprestatie'],
      dtype='object')


In [83]:
print(enddf.shape)
enddf.head()

(750, 11)


,goldLabel,Geslacht,Heb je eerder geroeid?,skollvoorroeien,1+ jaar al geroeid,Hoe vaak kan roeien,roeimotivatie,hoeveel sporten vroeger beoefend,# van plan te gaan roeien,# al geroeid,sportroeiprestatie
0,1,0,0,0,0,14,1,2.0,3.5,0.0,0
1,1,0,0,0,0,7,1,0.0,2.5,0.0,0
2,0,0,0,0,0,16,1,2.0,2.5,0.0,0
3,0,0,1,0,1,20,0,1.0,3.5,4.5,1
4,0,1,0,0,0,17,1,1.0,2.5,0.0,0


In [84]:
X, y = enddf.iloc[:,1:], enddf.iloc[:,0]
Xfem, yfem = dffemale.iloc[:,1:], dffemale.iloc[:,0]
Xmale, ymale = dfmale.iloc[:,1:], dfmale.iloc[:,0]


data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
ymale

4      0
8      1
11     0
12     0
15     0
      ..
737    1
738    1
739    1
746    1
747    1
Name: goldLabel, Length: 291, dtype: int32

In [89]:
def xgboost(X, y):  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    xg_reg = xgb.XGBClassifier(verbosity = 0)
    xg_reg.fit(X_train,y_train)

    preds = xg_reg.predict(X_test)
    # classifiedpreds = [1 if x > 0.5000 else 0 for x in preds ] 
#     f1_score(y_test, preds)
    scores = cross_validate(xg_reg, X, y, cv=5,scoring='f1',return_train_score=True)
    return np.mean(scores['test_score'])
def neighbor(X, y):
    X.fillna(0, inplace = True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    clfneigh = KNeighborsClassifier()
    clfneigh.fit(X_train, y_train)
    scores = cross_validate(clfneigh, X, y, cv=5,scoring='f1',return_train_score=True)
    return np.mean(scores['test_score'])

def tree(X, y):
    X.fillna(0, inplace = True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    clftree = DecisionTreeClassifier()
    clftree.fit(X_train, y_train)
    scores = cross_validate(clftree, X, y, cv=5,scoring='f1',return_train_score=True)
    return np.mean(scores['test_score'])

def linear(X, y):
    X.fillna(0, inplace = True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    clflinear = svm.SVC()
    clflinear.fit(X_train, y_train)
    scores = cross_validate(clflinear, X, y, cv=5,scoring='f1',return_train_score=True)
    return np.mean(scores['test_score'])

In [90]:
print('xgboost:\t',xgboost(X, y))
print('Neighbor:\t',neighbor(X, y))
print('Tree:\t\t',tree(X, y))
print('Tree:\t\t',linear(X, y))

C:\Users\rvendrig\AppData\Local\Continuum\anaconda3\envs\virtenv\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\rvendrig\AppData\Local\Continuum\anaconda3\envs\virtenv\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\rvend

xgboost:	 0.5712414530851908
Neighbor:	 0.4433873573808814
Tree:		 0.4525587039223403
Tree:		 0.3885024640352859


In [91]:
print('xgboost:\t',xgboost(Xfem, yfem), '\t', xgboost(Xmale, ymale))
print('Neighbor:\t',neighbor(Xfem, yfem), '\t', neighbor(Xmale, ymale))
print('Tree:\t\t',tree(Xfem, yfem), '\t', tree(Xmale, ymale))
print('Tree:\t\t',linear(Xfem, yfem), '\t', linear(Xmale, ymale))

C:\Users\rvendrig\AppData\Local\Continuum\anaconda3\envs\virtenv\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\rvendrig\AppData\Local\Continuum\anaconda3\envs\virtenv\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\rvend

xgboost:	 0.5201843131426791 	 0.7146405228758169
Neighbor:	 0.4228725038402458 	 0.5488089958821666
Tree:		 0.5512556388028086 	 0.6789108476064999
Tree:		 0.2171630015108276 	 0.5146869222731292


In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
xg_reg = xgb.XGBRegressor(objective ='binary:logistic')
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)
# np.set_printoptions(suppress=True)
preds


array([6.9192178e-03, 9.3767422e-01, 1.1467934e-01, 9.3736649e-01,
       9.8748350e-01, 6.0169179e-02, 6.3969478e-02, 9.1445833e-01,
       1.7718473e-01, 1.7762205e-02, 9.1445833e-01, 6.7519187e-03,
       9.9167144e-01, 7.2614523e-04, 5.3222454e-01, 2.7561348e-02,
       7.4684486e-02, 9.8604959e-01, 2.1078086e-02, 4.1118339e-02,
       1.7718473e-01, 3.9425558e-01, 4.0575583e-02, 9.2921394e-01,
       9.8069257e-01, 4.3794963e-01, 4.9070588e-01, 1.7718473e-01,
       9.6653640e-01, 8.4614277e-01, 8.5185951e-01, 1.0539672e-02,
       3.5653752e-01, 1.0728332e-01, 1.7718473e-01, 9.9041170e-01,
       3.5653752e-01, 9.3736649e-01, 1.4623969e-02, 9.5792449e-01,
       5.1298255e-01, 2.0223689e-01, 9.8748350e-01, 3.9790099e-04,
       1.3198276e-03, 7.2137809e-01, 1.7095637e-01, 1.5730508e-02,
       6.3969478e-02, 1.1562182e-02, 9.8069257e-01, 1.1562182e-02,
       1.2986003e-01, 4.0575583e-02, 4.3580788e-01, 9.3705666e-01,
       1.7718473e-01, 3.8507190e-02, 8.5185951e-01, 3.7020401e

In [93]:
xg_reg = xgb.XGBClassifier()
xg_reg.fit(Xfem,yfem)
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()


C:\Users\rvendrig\AppData\Local\Continuum\anaconda3\envs\virtenv\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ImportError: You must install matplotlib to plot importance

Todo:
- 2 modellen maken voor Geslacht?
- genormaliseerd vs niet hebben ong zelfde resultaat, uitvinden welke de beste is

In [94]:
path = r'C:\Users\rvendrig\OneDrive - Deloitte (O365D)\Desktop\TechProjects\WedstrijdRoeiVoorspeller\test_set' # use your path
all_files = glob.glob(os.path.join(path, "*.xls"))
all_files1 = [f.replace(r".xls",".txt") for f in all_files]  ## replacing the file extension with .txt
df_from_each_file = (pd.read_csv(f, delimiter = "\t") for f in all_files) ## reading the .txt files using csv reader
inputdf = pd.concat(df_from_each_file, ignore_index=True)
inputdf = inputdf.drop(['Geboortedatum','E-mailadres', 'Wat studeer je?','Telefoonnummer', 'Studentnummer'], axis = 1) # privacy sensitive

inputdf = drop_private_cols(inputdf)
inputdf = transform_df(inputdf)
inputdf.fillna(0, inplace = True)
inputdf = inputdf.drop(['Lengte (in cm)','Gewicht (in kg)'],axis = 1) # gewicht and lengte appears to work negatively on the prediction score

X_male_test_w_name = inputdf[inputdf['Geslacht'] == 1].drop(['Geslacht'], axis=1)
X_female_test_w_name = inputdf[inputdf['Geslacht'] == 0].drop(['Geslacht'], axis=1)
X_male_test = X_male_test_w_name.drop(['Naam'], axis = 1)
X_female_test = X_female_test_w_name.drop(['Naam'], axis = 1)

xg_regmale = xgb.XGBRegressor(objective ='binary:logistic')
xg_regfemale = xgb.XGBRegressor(objective ='binary:logistic')
xg_regmale.fit(Xmale,ymale)
xg_regfemale.fit(Xfem,yfem)

predmale = xg_regmale.predict(X_male_test)
predfemale = xg_regfemale.predict(X_female_test)

X_male_test_w_name['prediction'] = np.around(predmale, 5)
X_female_test_w_name['prediction'] = np.around(predfemale, 5)
test_set = pd.concat([X_male_test_w_name, X_female_test_w_name])
# test_set = test_set.loc[:, ['Naam','prediction']]
# test_set.sort_values(by='prediction', ascending = False, inplace = True)
test_set

,Naam,Heb je eerder geroeid?,skollvoorroeien,1+ jaar al geroeid,Hoe vaak kan roeien,roeimotivatie,hoeveel sporten vroeger beoefend,# van plan te gaan roeien,# al geroeid,sportroeiprestatie,prediction
8,Dani Fung-A-Jou,0,0,0,20,1,1.0,1.0,0.0,0,0.05526
10,Camille Arnts,0,0,0,15,1,1.0,2.5,0.0,0,0.18567
11,Jorre Lambregtse,0,0,0,20,1,2.0,2.5,0.0,0,0.88457
12,Wesley te Biesebeek,0,1,0,20,1,4.0,2.5,0.0,0,0.39194
13,Tijn de Pleijt,0,1,0,10,1,4.0,3.5,0.0,0,0.10453
...,...,...,...,...,...,...,...,...,...,...,...
125,Mathilde Verster,1,1,1,5,1,3.0,5.0,2.5,1,0.98960
127,Daphne Huyghe,0,0,0,20,1,2.0,3.5,0.0,0,0.82991
128,Laura te Rijdt,0,1,0,9,1,6.0,3.5,0.0,0,0.29276
130,Floor Lubeck,1,0,0,20,0,2.0,0.0,6.0,0,0.13447


In [44]:
boot = pd.read_csv('raw_data/people_with_boats.csv')
print(boot.shape)
boot['Boten'] = boot['Boten'].str.split(',')
boot = boot[boot['Boten'].str.len() != 1]
boot.dropna(axis=0, inplace = True)
boot.columns

(1114, 4)


Index(['Name', 'SaxoId', 'WpId', 'Boten'], dtype='object')

In [45]:
train = pd.read_csv('raw_data/training_data.csv')
print(train.shape)

(2769, 29)


In [46]:
train

,Naam,Geslacht,Welke sporten heb je beoefend?,"Anders, namelijk",Hoeveel jaar heb je je langst beoefende sport beoefend?,Wat is het fanatiekst dat je hebt gesport?,Wat is je beste sportprestatie?,Heb je eerder geroeid?,"Zo ja, hoe lang?","Zo ja, hoe vaak per week?",...,Instituut,Wanneer ben je begonnen met je studie?,Wanneer verwacht je af te studeren?,Heb je je zwemdiploma A?,"Indien je niet in Amsterdam woont, ben je op zoek naar een kamer in Amsterdam?","Op Skøll eten veel mensen vegetarisch, pas je voorkeur hieronder aan als je toch wel vlees wilt eten",Heb je medische bijzonderheden waar wij rekening mee moeten houden?,Namelijk,Heb je allergieën waar wij rekening mee moeten houden?,Wanneer kun je roeien?
0,7864534534540684764,Vrouw,"rowing, tennis, other",Ballet,Ballet-6 jaar Tennis-11 jaar,NaN,Meervoudig clubkampioen tennis,Ja,1/2 jaar,Minder dan 2 keer per week,...,NaN,NaN,NaN,Nee,n.v.t.,NaN,Nee,NaN,NaN,"ja ja ja ja, ja ja ja ja, ja ja ja ja, ja ja ja ja, ja ja ne ne"
1,5419786849097829421,Man,"atletics, ice-skating, swimming","Kitesurfen, snowboarden",10 jaar,NaN,Dagen lang snowboarden en kitesurfen,Nee,NaN,? keer per week,...,Hogeschool van Amsterdam,NaN,NaN,Nee,n.v.t.,NaN,Nee,NaN,NaN,"ne ja ja ja, ne ja ja ja, ne ja ja ja, ne ja ja ja, ne ja ja ja"
2,-6117707888134818157,Vrouw,other,Turnen en paardrijden,4 jaar,NaN,2x per week groepslessen bij sportcentrum VU,Nee,NaN,? keer per week,...,Vrije Universiteit Amsterdam,NaN,NaN,Nee,Nee,NaN,Nee,NaN,NaN,"ja ja ja ja, ja ja ja ja, ja ja ja ja, ne ne ne ne, ne ja ja ja"
3,-8496796740073669065,Vrouw,other,Dansen en turnen,6 jaar,NaN,NaN,Nee,NaN,? keer per week,...,NaN,NaN,NaN,Nee,n.v.t.,NaN,Nee,NaN,NaN,"ja ja ja ja, ne ne ja ja, ne ja ja ja, ne ne ja ja, ja ja ja ja"
4,-4555060004506103813,Vrouw,other,"Turnen, hardlopen en dansen",In totaal 6 jaar,NaN,"Deze vakantie heb ik een berg beklommen waarbij de route veel zwaarder bleek te zijn dan verwacht. Als kind heb ik vaker bergtochten gemaakt met mijn ouders dus ik ben in die zin wel wat gewend, maar deze tocht had ik onderschat. Omdat het die dag ook nog eens super warm was, heb ik vaak het idee gehad om er mee te stoppen en terug te gaan. Toen ik na lang doorzetten toch op de top van de berg uitkwam gaf dat een heel goed en voldaan gevoel.",Ja,Een keer om het uit te proberen,? keer per week,...,NaN,NaN,NaN,Nee,n.v.t.,NaN,Nee,NaN,NaN,"ja ja ja ja, ja ja ja ja, ja ja ja ja, ne ne ja ja, ja ja ja ja"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764,2795588844708659703,Man,"ice-skating, tennis, swimming",-,Schaatsen 3 jaar; Zwemmen 2 jaar; Tennis 2 jaar,NaN,"Hardloopwedstrijd, 5,2 km in 28 minuten",Nee,NaN,? keer per week,...,Universiteit van Amsterdam,NaN,NaN,Nee,n.v.t.,NaN,Nee,NaN,NaN,"ja ja ja ja, ja ja ja ja, ja ja ne ne, ja ja ja ja, ja ja ja ja"
2765,8948710549763942018,Vrouw,"rowing, atletics, hockey",NaN,"Roeien 2 jaar, hockey 7 jaar, atletiek 2 jaar.",NaN,Kampioen worden met mijn hockeyteam.,Ja,2 jaar,Minder dan 2 keer per week,...,NaN,NaN,NaN,Nee,Ja,NaN,Nee,NaN,NaN,"ne ja ja ja, ja ja ja ja, ja ja ja ja, ne ne ja ja, ne ne ja ja"
2766,-6552179247405681344,Man,soccer,NaN,10 jaar,NaN,In een voetbalteam spelen die eerste klasse voetbalde en boven in mee kon draaien,Nee,NaN,? keer per week,...,NaN,NaN,NaN,Nee,n.v.t.,NaN,Nee,NaN,NaN,"ja ja ja ja, ja ja ja ja, ja ja ja ja, ja ja ja ja, ja ja ja ja"
2767,2929104241939964515,Vrouw,"swimming, other","Spinning, hardlopen en HIIT",Vanaf 2013,NaN,"Ik heb tot nu toe nog geen echte sportprestatie neergezet, maar als ik sport probeer ik mezelf steeds te verbeteren op gebied van kracht, conditie en behendigheid. Het lijkt me wel heel leuk om te trainen voor prestaties met het roeien!",Nee,NaN,? keer per week,...,NaN,NaN,NaN,Nee,n.v.t.,NaN,Nee,NaN,NaN,"ne ne ne ne, ne ja ja ne, ja ja ja ne, ja ja ja ne, ja ja ja ne"
